# Regression Model A: Field 3

## Install libraries

In [ ]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [ ]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

## Import data

In [ ]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 3 (F3)
Call the data according to the Field. 

In [ ]:
dE1=d1[which(d1$`ESTATE CODE`=="E3" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E3" ),]
dE4=d4[which(d4$Estate=="E3" ),]
dE5=d5[which(d5$Estate=="E3" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [ ]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [ ]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Train and test data

Split the dataset into train and test sets. 

After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

We use the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 

In [ ]:
train = dg1[dg1$year<=2016,]
test = dg1[dg1$year==2017,]
dim(train)
dim(test)

## Regression

Fit the data using multiple regression for all variables. 

In [ ]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [ ]:
anova(a1)

### Using stepwise

In [ ]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of the model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [ ]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (R-square) to know how close the data to the fitted regression line. 

In [ ]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(a1)$r.squared)
fit